In [1]:
#Importing necessary libraries that has ready to use functions:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import folium
import io
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QMessageBox, QHBoxLayout
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtCore import QUrl, pyqtSlot

#Reading the data sataset and assigning features:
df = pd.read_csv(r'C:\Users\zayca\1km_kocaeli_provice_location_counts.csv')
features = df[['Mall_Count', 'Charging_Station_Count', 'Gas_Station_Count', 'Car_Park_Count', 'Kocaeli_Kopark_Count', 'Kocaeli_Freecarpark_Count', 'Hospital_Count', 'Government_Office_Count', 'Traffic_Sensors_Count', 'Highway_Count']]

#Declaring sclaer functions to automate modeling process:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

#Declaring and fitting kmeans to the scaled features to start clustering:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(features_scaled)

#Defining the haversine function to calculate the distance between 2 points:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371
    return c * r

#Defining count_facilities function to get the number of facilities near the points:
def count_facilities(df, lon, lat, radius=1):
    distances = haversine(lon, lat, df['Longitude'].to_numpy(), df['Latitude'].to_numpy())
    count = np.sum(distances < radius)
    return str(count)

#Reading facility datasets:
mall_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_mall.csv')
charging_station_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_charging_station.csv')
gas_station_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_gas_station.csv')
car_park_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_outdoor_car_park.csv')
kocaeli_kopark_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_kopark.csv')
kocaeli_freecarpark_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_freecarpark.csv')
hospital_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_hospital.csv')
government_office_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_government_office.csv')
traffic_sensors_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_traffic_sensors.csv')
kocaeli_highway_df = pd.read_csv(r'C:\Users\zayca\brisa\kocaeli_highway.csv')

#Definin predict_cluster function to start predicting with scaled features declared above. The result gives the cluster.
def predict_cluster(lon, lat):
    counts = {
        'Mall Count': count_facilities(mall_df, lon, lat),
        'Charging Station Count': count_facilities(charging_station_df, lon, lat),
        'Gas Station Count': count_facilities(gas_station_df, lon, lat),
        'Car Park Count': count_facilities(car_park_df, lon, lat),
        'Kocaeli Kopark Count': count_facilities(kocaeli_kopark_df, lon, lat),
        'Kocaeli Freecarpark Count': count_facilities(kocaeli_freecarpark_df, lon, lat),
        'Hospital Count': count_facilities(hospital_df, lon, lat),
        'Government Office Count': count_facilities(government_office_df, lon, lat),
        'Traffic Sensors Count': count_facilities(traffic_sensors_df, lon, lat),
        'Highway Count': count_facilities(kocaeli_highway_df, lon, lat)
    }

    features = np.array(list(counts.values()), dtype=float).reshape(1, -1)
    features_scaled = scaler.transform(features)
    cluster = kmeans.predict(features_scaled)
    return cluster[0], counts

#Creating map to add to the display with 9 points that are comging from the mathematical model:
def create_map(lat=None, lon=None, cluster=None):
    m = folium.Map(location=[40.8533, 29.8815], zoom_start=10)
    m.add_child(folium.LatLngPopup())

    points = [
        (40.76509, 29.89784),
        (40.80028, 29.45069),
        (40.80302, 29.37917),
        (40.76566, 29.90921),
        (40.75417, 29.94951),
        (40.7504, 29.95982),
        (40.7123, 29.84105),
        (40.70737, 29.88444),
        (40.70931, 29.82254)
    ]

    #Adding these 9 points to the map as default.
    for point in points:
        folium.CircleMarker(
            location=[point[0], point[1]],
            radius=7,
            color='red',
            fill=True,
            fill_color='red'
        ).add_to(m)
    
    #Cluster classification is done here.
    if lat is not None and lon is not None and cluster is not None:
        classification = "Bad"
        if cluster == 2:
            classification = "Good"
        elif cluster == 1:
            classification = "Moderate"

        #By using folium library, markers are added on the map.
        folium.Marker(
            location=[lat, lon],
            popup=f"New Point, Cluster: {cluster}, Classification: {classification}",
            icon=folium.Icon(color='blue' if cluster == 0 else 'green' if cluster == 1 else 'red')
        ).add_to(m)

    #Saving the map into the file to access later.
    map_data = io.BytesIO()
    m.save(map_data, close_file=False)
    return map_data.getvalue().decode()

#Declaring the class of the application. PyQT library allows this class to be used to create a desktop application.
class App(QWidget):
    def __init__(self):
        super().__init__()
        self.selected_lat = None
        self.selected_lng = None
        self.initUI()
        
    #Defining initUI to create the UI of the display. Frontend part of the display is created and added in this part.
    def initUI(self):
        self.setWindowTitle('Cluster Prediction')
        self.setGeometry(100, 100, 1000, 700)
        self.setStyleSheet("background-color: #96B9DC;")

        layout = QVBoxLayout()

        header_layout = QHBoxLayout()
        header_label = QLabel('Ev Charging Stations Project')
        header_label.setStyleSheet("color: #2D5986; font-size: 24px;")
        header_layout.addWidget(header_label)
        header_layout.addStretch()

        layout.addLayout(header_layout)

        self.lat_label = QLabel('Enter Latitude:')
        layout.addWidget(self.lat_label)
        self.lat_input = QLineEdit(self)
        layout.addWidget(self.lat_input)

        self.lon_label = QLabel('Enter Longitude:')
        layout.addWidget(self.lon_label)
        self.lon_input = QLineEdit(self)
        layout.addWidget(self.lon_input)

        self.predict_button = QPushButton('Predict Cluster', self)
        self.predict_button.setStyleSheet("background-color: #2D5986; color: white; font-size: 18px; padding: 10px;")
        self.predict_button.clicked.connect(self.get_cluster)
        layout.addWidget(self.predict_button)

        self.map_view = QWebEngineView()
        layout.addWidget(self.map_view)

        self.setLayout(layout)
        self.load_map()

    #Defining get_cluster function to embed the model automatically. Text inputs are given in this part.
    def get_cluster(self):
        lat = self.lat_input.text()
        lon = self.lon_input.text()
        if lat and lon:
            try:
                lat = float(lat)
                lon = float(lon)
                self.selected_lat = lat
                self.selected_lng = lon
                cluster, counts = predict_cluster(lon, lat)
                counts_text = '\n'.join([f"{key}: {value}" for key, value in counts.items()])

                classification = "Bad"
                if cluster == 2:
                    classification = "Good"
                elif cluster == 1:
                    classification = "Moderate"
                
                #Pop-up box is created here with one of the PyQT functions: QMessageBox.
                msg_box = QMessageBox(self)
                msg_box.setStyleSheet("background-color: #96B9DC;")
                msg_box.information(self, "Cluster Result", f"Classification: {classification}\n\nCounts:\n{counts_text}")
                self.update_map(lat, lon, cluster)
            except ValueError:
                msg_box = QMessageBox(self)
                msg_box.setStyleSheet("background-color: #96B9DC;")
                msg_box.critical(self, "Invalid Input", "Please enter valid latitude and longitude values.")

    #Map is loaded into javascript. Javascript connects to the app itself to embed content above.
    def load_map(self):
        map_html = create_map()
        self.map_view.setHtml(map_html)
        self.map_view.page().runJavaScript(
            "document.querySelector('.leaflet-container').addEventListener('click', function(e) { "
            "    var popup = document.querySelector('.leaflet-popup-content'); "
            "    if (popup) { window.pyqtSlot.getCoordinates(popup.innerText); }"
            "});"
        )
    
    #Map is updated here. With each action made on the map, the action is recognized here.
    def update_map(self, lat, lon, cluster):
        map_html = create_map(lat, lon, cluster)
        self.map_view.setHtml(map_html)
        self.map_view.page().runJavaScript(
            "document.querySelector('.leaflet-container').addEventListener('click', function(e) { "
            "    var popup = document.querySelector('.leaflet-popup-content'); "
            "    if (popup) { window.pyqtSlot.getCoordinates(popup.innerText); }"
            "});"
        )

    #Lat and long values are taken seperately in here to getting put in the slot. Slot allows application to work in backend.
    @pyqtSlot(str)
    def getCoordinates(self, coordinates):
        try:
            lat, lon = map(float, coordinates.split('\n'))
            self.selected_lat = lat
            self.selected_lng = lon
            cluster, counts = predict_cluster(lon, lat)
            counts_text = '\n'.join([f"{key}: {value}" for key, value in counts.items()])

            classification = "Bad"
            if cluster == 2:
                classification = "Good"
            elif cluster == 1:
                classification = "Moderate"

            self.update_map(lat, lon, cluster)
            msg_box = QMessageBox(self)
            msg_box.setStyleSheet("background-color: #96B9DC;")
            msg_box.information(self, "Cluster Result", f"Classification: {classification}\n\nCounts:\n{counts_text}")
        except Exception as e:
            print(f"Error: {e}")

#Applications system is declared in this part. Acitons of opening and existing the app happens here.
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    ex.show()
    sys.exit(app.exec_())


C:\Users\zayca\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


SystemExit: 0

C:\Users\zayca\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
